In [8]:
import os
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from dataset import *

In [9]:
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader

In [10]:
learning_rate=1e-5
batch_size=1
num_epochs=40
num_classes=7

In [11]:
#training data test data loading
train_dirr="D:/emotion_dataset/mmidataset/VideoWithImageLabels/face_detected/train/labels.csv"
train_dataset=ImageDataset(csv_file=train_dirr
                           ,transform=ToTensor())
train_loader=DataLoader(train_dataset,batch_size=1,shuffle=False)

test_dirr="D:/emotion_dataset/mmidataset/VideoWithImageLabels/face_detected/test/labels.csv"
test_dataset=ImageDataset(csv_file=test_dirr
                           ,transform=ToTensor())
test_loader=DataLoader(test_dataset,batch_size=1,shuffle=False)


In [12]:
#model
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet,self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1),
                                nn.BatchNorm2d(64),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=3,stride=2))
        
        #res2a
        self.res2a_branch1=nn.Sequential(nn.Conv2d(64,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256))
        self.res2a_branch2a=nn.Sequential(nn.Conv2d(64,64,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU())
        self.res2a_branch2b=nn.Sequential(nn.Conv2d(64,64,kernel_size=3,padding=1,stride=1),
                                         nn.ReLU())
        self.res2a_branch2c=nn.Sequential(nn.Conv2d(64,256,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(256))
        
        #res2b
        self.res2b_branch2a=nn.Sequential(nn.Conv2d(512,64,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU())
        self.res2b_branch2b=nn.Sequential(nn.Conv2d(64,64,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU())
        self.res2b_branch2c=nn.Sequential(nn.Conv2d(64,256,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(256))
        
        #res2c
        self.res2c_branch2a=nn.Sequential(nn.Conv2d(768,64,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU())
        self.res2c_branch2b=nn.Sequential(nn.Conv2d(64,64,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU())
        self.res2c_branch2c=nn.Sequential(nn.Conv2d(64,256,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(256))
        
        #res3a
        self.res3a_branch1=nn.Sequential(nn.Conv2d(1024,512,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(512))
        self.res3a_branch2a=nn.Sequential(nn.Conv2d(1024,128,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3a_branch2b=nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3a_branch2c=nn.Sequential(nn.Conv2d(128,512,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(512))
        
        #res3b
        self.res3b_branch2a=nn.Sequential(nn.Conv2d(1024,128,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3b_branch2b=nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3b_branch2c=nn.Sequential(nn.Conv2d(128,512,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(512))
        
        #res3c
        self.res3c_branch2a=nn.Sequential(nn.Conv2d(1536,128,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3c_branch2b=nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3c_branch2c=nn.Sequential(nn.Conv2d(128,512,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(512))
        
        #res3d
        self.res3d_branch2a=nn.Sequential(nn.Conv2d(2048,128,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3d_branch2b=nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU())
        self.res3d_branch2c=nn.Sequential(nn.Conv2d(128,512,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(512))
        
        #res4a
        self.res4a_branch1=nn.Sequential(nn.Conv2d(2560,1024,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(1024))
        self.res4a_branch2a=nn.Sequential(nn.Conv2d(2560,256,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4a_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4a_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        
        #res4b
        self.res4b_branch2a=nn.Sequential(nn.Conv2d(2048,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4b_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4b_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        #res4c
        self.res4c_branch2a=nn.Sequential(nn.Conv2d(3072,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4c_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4c_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        
        #res4d
        self.res4d_branch2a=nn.Sequential(nn.Conv2d(4096,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4d_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4d_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        
        #res4e
        self.res4e_branch2a=nn.Sequential(nn.Conv2d(5120,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4e_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4e_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        
        #res4f
        self.res4f_branch2a=nn.Sequential(nn.Conv2d(6144,256,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4f_branch2b=nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU())
        self.res4f_branch2c=nn.Sequential(nn.Conv2d(256,1024,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(1024))
        
        #res5a
        self.res5a_branch1=nn.Sequential(nn.Conv2d(7168,2048,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(2048))
        self.res5a_branch2a=nn.Sequential(nn.Conv2d(7168,512,kernel_size=1,stride=2,padding=0),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5a_branch2b=nn.Sequential(nn.Conv2d(512,512,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5a_branch2c=nn.Sequential(nn.Conv2d(512,2048,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(2048))
        #res5b
        self.res5b_branch2a=nn.Sequential(nn.Conv2d(4096,512,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5b_branch2b=nn.Sequential(nn.Conv2d(512,512,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5b_branch2c=nn.Sequential(nn.Conv2d(512,2048,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(2048))
        
        #res5c
        self.res5c_branch2a=nn.Sequential(nn.Conv2d(6144,512,kernel_size=1,stride=1,padding=0),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5c_branch2b=nn.Sequential(nn.Conv2d(512,512,kernel_size=3,padding=1,stride=1),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU())
        self.res5c_branch2c=nn.Sequential(nn.Conv2d(512,2048,kernel_size=1,padding=0,stride=1),
                                         nn.BatchNorm2d(2048))
        
        
        #res2c_new
        self.res2c_new=nn.Sequential(nn.Conv2d(1024,512,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(),
                                    nn.Conv2d(512,1024,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(),
                                    nn.Conv2d(1024,2048,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(),
                                    nn.AvgPool2d(kernel_size=3,stride=1),
                                    nn.Conv2d(2048,64,kernel_size=1,stride=1,padding=0))
        #res3c_new
        self.res3d_new=nn.Sequential(nn.Conv2d(2560,1024,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(),
                                    nn.Conv2d(1024,2048,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(),
                                    nn.AvgPool2d(kernel_size=3,stride=1),
                                    nn.Conv2d(2048,64,kernel_size=1,stride=1,padding=0))
        
        #res4f_new
        self.res4f_new=nn.Sequential(nn.Conv2d(7168,2048,kernel_size=3,stride=2,padding=1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(),
                                    nn.AvgPool2d(kernel_size=3,stride=1),
                                    nn.Conv2d(2048,64,kernel_size=1,stride=1,padding=0))
        #res4f_new
        self.res5c_new=nn.Sequential(nn.AvgPool2d(kernel_size=3,stride=1),
                                     nn.Conv2d(8192,64,kernel_size=1,stride=1,padding=0))
        
        self.relu=nn.ReLU()
        self.fc1=nn.Linear(256,num_classes)
        self.fc2=nn.Linear(num_classes*4,num_classes)
        
        
        
    def forward(self,x):
        
        conv1=self.conv1(x)
        
        #res2a
        res2a_a=self.res2a_branch1(conv1)
        res2a_b=self.res2a_branch2a(conv1)
        res2a_b=self.res2a_branch2b(res2a_b)
        res2a_b=self.res2a_branch2c(res2a_b)
        res2a_out=torch.cat((res2a_a,res2a_b),1)
        res2a_out=self.relu(res2a_out)
        
        #res2b
        res2b_b=self.res2b_branch2a(res2a_out)
        res2b_b=self.res2b_branch2b(res2b_b)
        res2b_b=self.res2b_branch2c(res2b_b)
        res2b_out=torch.cat((res2a_out,res2b_b),1)
        res2b_out=self.relu(res2b_out)
        
        #res2c
        res2c_b=self.res2c_branch2a(res2b_out)
        res2c_b=self.res2c_branch2b(res2c_b)
        res2c_b=self.res2c_branch2c(res2c_b)
        res2c_out=torch.cat((res2b_out,res2c_b),1)
        res2c_out=self.relu(res2c_out)
        
        #res3a
        res3a_a=self.res3a_branch1(res2c_out)
        res3a_b=self.res3a_branch2a(res2c_out)
        res3a_b=self.res3a_branch2b(res3a_b)
        res3a_b=self.res3a_branch2c(res3a_b)
        res3a_out=torch.cat((res3a_a,res3a_b),1)
        res3a_out=self.relu(res3a_out)
        
        #res3b
        res3b_b=self.res3b_branch2a(res3a_out)
        res3b_b=self.res3b_branch2b(res3b_b)
        res3b_b=self.res3b_branch2c(res3b_b)
        res3b_out=torch.cat((res3a_out,res3b_b),1)
        res3b_out=self.relu(res3b_out)
        
        #res3c
        res3c_b=self.res3c_branch2a(res3b_out)
        res3c_b=self.res3c_branch2b(res3c_b)
        res3c_b=self.res3c_branch2c(res3c_b)
        res3c_out=torch.cat((res3b_out,res3c_b),1)
        res3c_out=self.relu(res3c_out)
        
        #res3d
        res3d_b=self.res3d_branch2a(res3c_out)
        res3d_b=self.res3d_branch2b(res3d_b)
        res3d_b=self.res3d_branch2c(res3d_b)
        res3d_out=torch.cat((res3c_out,res3d_b),1)
        res3d_out=self.relu(res3d_out)
        
        #res4a
        res4a_a=self.res4a_branch1(res3d_out)
        res4a_b=self.res4a_branch2a(res3d_out)
        res4a_b=self.res4a_branch2b(res4a_b)
        res4a_b=self.res4a_branch2c(res4a_b)
        res4a_out=torch.cat((res4a_a,res4a_b),1)
        res4a_out=self.relu(res4a_out)
        
        #res4b
        res4b_b=self.res4b_branch2a(res4a_out)
        res4b_b=self.res4b_branch2b(res4b_b)
        res4b_b=self.res4b_branch2c(res4b_b)
        res4b_out=torch.cat((res4a_out,res4b_b),1)
        res4b_out=self.relu(res4b_out)
        
        #res4c
        res4c_b=self.res4c_branch2a(res4b_out)
        res4c_b=self.res4c_branch2b(res4c_b)
        res4c_b=self.res4c_branch2c(res4c_b)
        res4c_out=torch.cat((res4b_out,res4c_b),1)
        res4c_out=self.relu(res4c_out)
        
        #res4d
        res4d_b=self.res4d_branch2a(res4c_out)
        res4d_b=self.res4d_branch2b(res4d_b)
        res4d_b=self.res4d_branch2c(res4d_b)
        res4d_out=torch.cat((res4c_out,res4d_b),1)
        res4d_out=self.relu(res4d_out)
        
        #res4e
        res4e_b=self.res4e_branch2a(res4d_out)
        res4e_b=self.res4e_branch2b(res4e_b)
        res4e_b=self.res4e_branch2c(res4e_b)
        res4e_out=torch.cat((res4d_out,res4e_b),1)
        res4e_out=self.relu(res4e_out)
        
        #res4f
        res4f_b=self.res4f_branch2a(res4e_out)
        res4f_b=self.res4f_branch2b(res4f_b)
        res4f_b=self.res4f_branch2c(res4f_b)
        res4f_out=torch.cat((res4e_out,res4f_b),1)
        res4f_out=self.relu(res4f_out)
        
        #res5a
        res5a_a=self.res5a_branch1(res4f_out)
        res5a_b=self.res5a_branch2a(res4f_out)
        res5a_b=self.res5a_branch2b(res5a_b)
        res5a_b=self.res5a_branch2c(res5a_b)
        res5a_out=torch.cat((res5a_a,res5a_b),1)
        res5a_out=self.relu(res5a_out)
        #res5b
        res5b_b=self.res5b_branch2a(res5a_out)
        res5b_b=self.res5b_branch2b(res5b_b)
        res5b_b=self.res5b_branch2c(res5b_b)
        res5b_out=torch.cat((res5a_out,res5b_b),1)
        res5b_out=self.relu(res5b_out)

        #res5c
        res5c_b=self.res5c_branch2a(res5b_out)
        res5c_b=self.res5c_branch2b(res5c_b)
        res5c_b=self.res5c_branch2c(res5c_b)
        res5c_out=torch.cat((res5b_out,res5c_b),1)
        res5c_out=self.relu(res5c_out)
        
        ##########################block############################
        #res2c_new (ss_block)
        res2c_new=self.res2c_new(res2c_out)
        res2c_new=res2c_new.reshape(res2c_new.size(0),-1)
#         print("res2c_new.shape:",res2c_new.shape)
        res2c_new=self.fc1(res2c_new)
        
        #res3d_new (is_block)
        res3d_new=self.res3d_new(res3d_out)
        res3d_new=res3d_new.reshape(res3d_new.size(0),-1)
        res3d_new=self.fc1(res3d_new)
        
        #res4f_new (is_block)
        res4f_new=self.res4f_new(res4f_out)
        res4f_new=res4f_new.reshape(res4f_new.size(0),-1)
        res4f_new=self.fc1(res4f_new)
        
        #res5c_new (ds_block)
        res5c_new=self.res5c_new(res5c_out)
        res5c_new=res5c_new.reshape(res5c_new.size(0),-1)
        res5c_new=self.fc1(res5c_new)
        
        concat_branch1=torch.cat((res2c_new,res3d_new,res4f_new,res5c_new),1)
        concat_branch2=self.fc2(concat_branch1)
        
        return res2c_new,res3d_new,res4f_new,res5c_new,concat_branch2

In [13]:
model=ResNet().cuda()
softmax=nn.Softmax()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# a=torch.rand((1,3,128,128))
# a=a.cuda()
# outputs=model(a)
# print("model_output : ",outputs.shape)

In [14]:
npz_x=[]
npz_loss=[]
#training step
total_step=len(train_loader)
for epoch in range(num_epochs):
    for i,(image,label) in enumerate(train_loader):
        image=image.cuda()
        label=label.cuda()
        ss_output,is_output1,is_output2,ds_output,output=model(image)
      
        ss_loss=criterion(ss_output,label)
        is_loss1=criterion(is_output1,label)
        is_loss2=criterion(is_output2,label)
        ds_loss=criterion(ds_output,label)
        final_loss=ss_loss+is_loss1+is_loss2+ds_loss+criterion(output,label)
        #backward and optimize
        
        optimizer.zero_grad()
        final_loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}],final_Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, final_loss.item()))
    if(epoch%3==0):
        #save training result
        torch.save(model.state_dict(), './save_data/total_model.ckpt')
        npz_x.append(epoch)
        npz_loss.append(final_loss)
        np.savez("./save_data/total_model_loss.npz",x=npz_x,y=npz_loss)
        
np.savez("./save_data/total_model_loss.npz",x=npz_x,y=npz_loss)
#save training result
torch.save(model.state_dict(), './save_data/total_model.ckpt')     

Epoch [1/20], Step [100/12299],final_Loss: 0.3929
Epoch [1/20], Step [200/12299],final_Loss: 8.6102
Epoch [1/20], Step [300/12299],final_Loss: 0.6233
Epoch [1/20], Step [400/12299],final_Loss: 2.0855
Epoch [1/20], Step [500/12299],final_Loss: 0.6122
Epoch [1/20], Step [600/12299],final_Loss: 0.8908
Epoch [1/20], Step [700/12299],final_Loss: 30.9837
Epoch [1/20], Step [800/12299],final_Loss: 0.9792
Epoch [1/20], Step [900/12299],final_Loss: 0.4410
Epoch [1/20], Step [1000/12299],final_Loss: 1.8991
Epoch [1/20], Step [1100/12299],final_Loss: 0.4905
Epoch [1/20], Step [1200/12299],final_Loss: 0.8468
Epoch [1/20], Step [1300/12299],final_Loss: 2.0638
Epoch [1/20], Step [1400/12299],final_Loss: 0.5117
Epoch [1/20], Step [1500/12299],final_Loss: 5.0574
Epoch [1/20], Step [1600/12299],final_Loss: 22.4239
Epoch [1/20], Step [1700/12299],final_Loss: 16.2866
Epoch [1/20], Step [1800/12299],final_Loss: 0.5029
Epoch [1/20], Step [1900/12299],final_Loss: 0.6788
Epoch [1/20], Step [2000/12299],final

Epoch [2/20], Step [3900/12299],final_Loss: 0.4180
Epoch [2/20], Step [4000/12299],final_Loss: 4.0955
Epoch [2/20], Step [4100/12299],final_Loss: 0.2690
Epoch [2/20], Step [4200/12299],final_Loss: 31.1882
Epoch [2/20], Step [4300/12299],final_Loss: 0.7614
Epoch [2/20], Step [4400/12299],final_Loss: 0.5774
Epoch [2/20], Step [4500/12299],final_Loss: 0.4494
Epoch [2/20], Step [4600/12299],final_Loss: 0.6919
Epoch [2/20], Step [4700/12299],final_Loss: 2.5945
Epoch [2/20], Step [4800/12299],final_Loss: 5.1873
Epoch [2/20], Step [4900/12299],final_Loss: 4.6677
Epoch [2/20], Step [5000/12299],final_Loss: 0.3845
Epoch [2/20], Step [5100/12299],final_Loss: 6.1813
Epoch [2/20], Step [5200/12299],final_Loss: 1.0403
Epoch [2/20], Step [5300/12299],final_Loss: 0.7258
Epoch [2/20], Step [5400/12299],final_Loss: 9.5191
Epoch [2/20], Step [5500/12299],final_Loss: 1.9649
Epoch [2/20], Step [5600/12299],final_Loss: 0.8405
Epoch [2/20], Step [5700/12299],final_Loss: 12.6274
Epoch [2/20], Step [5800/1229

Epoch [3/20], Step [7700/12299],final_Loss: 4.1975
Epoch [3/20], Step [7800/12299],final_Loss: 29.4147
Epoch [3/20], Step [7900/12299],final_Loss: 0.4882
Epoch [3/20], Step [8000/12299],final_Loss: 0.5906
Epoch [3/20], Step [8100/12299],final_Loss: 0.5432
Epoch [3/20], Step [8200/12299],final_Loss: 2.9443
Epoch [3/20], Step [8300/12299],final_Loss: 0.4942
Epoch [3/20], Step [8400/12299],final_Loss: 0.8712
Epoch [3/20], Step [8500/12299],final_Loss: 0.1909
Epoch [3/20], Step [8600/12299],final_Loss: 0.4345
Epoch [3/20], Step [8700/12299],final_Loss: 0.2148
Epoch [3/20], Step [8800/12299],final_Loss: 0.2136
Epoch [3/20], Step [8900/12299],final_Loss: 0.3971
Epoch [3/20], Step [9000/12299],final_Loss: 20.4474
Epoch [3/20], Step [9100/12299],final_Loss: 3.2344
Epoch [3/20], Step [9200/12299],final_Loss: 0.9314
Epoch [3/20], Step [9300/12299],final_Loss: 0.6721
Epoch [3/20], Step [9400/12299],final_Loss: 12.2693
Epoch [3/20], Step [9500/12299],final_Loss: 0.3015
Epoch [3/20], Step [9600/122

Epoch [4/20], Step [11400/12299],final_Loss: 0.3477
Epoch [4/20], Step [11500/12299],final_Loss: 0.5691
Epoch [4/20], Step [11600/12299],final_Loss: 0.4835
Epoch [4/20], Step [11700/12299],final_Loss: 0.2262
Epoch [4/20], Step [11800/12299],final_Loss: 6.1421
Epoch [4/20], Step [11900/12299],final_Loss: 0.2681
Epoch [4/20], Step [12000/12299],final_Loss: 0.8434
Epoch [4/20], Step [12100/12299],final_Loss: 0.2764
Epoch [4/20], Step [12200/12299],final_Loss: 0.3379
Epoch [5/20], Step [100/12299],final_Loss: 0.2591
Epoch [5/20], Step [200/12299],final_Loss: 4.6823
Epoch [5/20], Step [300/12299],final_Loss: 0.2501
Epoch [5/20], Step [400/12299],final_Loss: 0.6221
Epoch [5/20], Step [500/12299],final_Loss: 0.2146
Epoch [5/20], Step [600/12299],final_Loss: 0.2056
Epoch [5/20], Step [700/12299],final_Loss: 23.3798
Epoch [5/20], Step [800/12299],final_Loss: 0.1759
Epoch [5/20], Step [900/12299],final_Loss: 0.1094
Epoch [5/20], Step [1000/12299],final_Loss: 0.4315
Epoch [5/20], Step [1100/12299

Epoch [6/20], Step [3000/12299],final_Loss: 0.5528
Epoch [6/20], Step [3100/12299],final_Loss: 0.9858
Epoch [6/20], Step [3200/12299],final_Loss: 0.7607
Epoch [6/20], Step [3300/12299],final_Loss: 7.7250
Epoch [6/20], Step [3400/12299],final_Loss: 0.2435
Epoch [6/20], Step [3500/12299],final_Loss: 0.2761
Epoch [6/20], Step [3600/12299],final_Loss: 1.2497
Epoch [6/20], Step [3700/12299],final_Loss: 0.4458
Epoch [6/20], Step [3800/12299],final_Loss: 0.3008
Epoch [6/20], Step [3900/12299],final_Loss: 0.2810
Epoch [6/20], Step [4000/12299],final_Loss: 4.1078
Epoch [6/20], Step [4100/12299],final_Loss: 0.3073
Epoch [6/20], Step [4200/12299],final_Loss: 34.6979
Epoch [6/20], Step [4300/12299],final_Loss: 0.9324
Epoch [6/20], Step [4400/12299],final_Loss: 0.6570
Epoch [6/20], Step [4500/12299],final_Loss: 0.3421
Epoch [6/20], Step [4600/12299],final_Loss: 0.5026
Epoch [6/20], Step [4700/12299],final_Loss: 2.4485
Epoch [6/20], Step [4800/12299],final_Loss: 3.8352
Epoch [6/20], Step [4900/12299

...

Epoch [15/20], Step [3700/12299],final_Loss: 0.5450
Epoch [15/20], Step [3800/12299],final_Loss: 0.2686
Epoch [15/20], Step [3900/12299],final_Loss: 0.3613
Epoch [15/20], Step [4000/12299],final_Loss: 4.3313
Epoch [15/20], Step [4100/12299],final_Loss: 0.3820
Epoch [15/20], Step [4200/12299],final_Loss: 27.5758
Epoch [15/20], Step [4300/12299],final_Loss: 1.0340
Epoch [15/20], Step [4400/12299],final_Loss: 0.6805
Epoch [15/20], Step [4500/12299],final_Loss: 0.4364
Epoch [15/20], Step [4600/12299],final_Loss: 0.5354
Epoch [15/20], Step [4700/12299],final_Loss: 1.5105
Epoch [15/20], Step [4800/12299],final_Loss: 4.7368
Epoch [15/20], Step [4900/12299],final_Loss: 5.9439
Epoch [15/20], Step [5000/12299],final_Loss: 0.3110
Epoch [15/20], Step [5100/12299],final_Loss: 5.5065
Epoch [15/20], Step [5200/12299],final_Loss: 0.9268
Epoch [15/20], Step [5300/12299],final_Loss: 0.7194
Epoch [15/20], Step [5400/12299],final_Loss: 7.5854
Epoch [15/20], Step [5500/12299],final_Loss: 1.6756
Epoch [15/2

Epoch [16/20], Step [7200/12299],final_Loss: 0.2529
Epoch [16/20], Step [7300/12299],final_Loss: 0.7186
Epoch [16/20], Step [7400/12299],final_Loss: 2.7800
Epoch [16/20], Step [7500/12299],final_Loss: 1.8750
Epoch [16/20], Step [7600/12299],final_Loss: 1.1943
Epoch [16/20], Step [7700/12299],final_Loss: 3.1917
Epoch [16/20], Step [7800/12299],final_Loss: 17.5734
Epoch [16/20], Step [7900/12299],final_Loss: 0.5262
Epoch [16/20], Step [8000/12299],final_Loss: 0.4006
Epoch [16/20], Step [8100/12299],final_Loss: 0.3899
Epoch [16/20], Step [8200/12299],final_Loss: 0.9592
Epoch [16/20], Step [8300/12299],final_Loss: 0.7822
Epoch [16/20], Step [8400/12299],final_Loss: 0.8128
Epoch [16/20], Step [8500/12299],final_Loss: 0.1824
Epoch [16/20], Step [8600/12299],final_Loss: 0.2906
Epoch [16/20], Step [8700/12299],final_Loss: 0.1296
Epoch [16/20], Step [8800/12299],final_Loss: 0.2497
Epoch [16/20], Step [8900/12299],final_Loss: 0.4938
Epoch [16/20], Step [9000/12299],final_Loss: 29.3797
Epoch [16/

Epoch [17/20], Step [10600/12299],final_Loss: 0.3110
Epoch [17/20], Step [10700/12299],final_Loss: 2.3739
Epoch [17/20], Step [10800/12299],final_Loss: 0.2373
Epoch [17/20], Step [10900/12299],final_Loss: 1.4205
Epoch [17/20], Step [11000/12299],final_Loss: 4.3432
Epoch [17/20], Step [11100/12299],final_Loss: 0.6547
Epoch [17/20], Step [11200/12299],final_Loss: 3.4599
Epoch [17/20], Step [11300/12299],final_Loss: 0.6316
Epoch [17/20], Step [11400/12299],final_Loss: 0.3532
Epoch [17/20], Step [11500/12299],final_Loss: 0.4699
Epoch [17/20], Step [11600/12299],final_Loss: 0.6010
Epoch [17/20], Step [11700/12299],final_Loss: 0.2611
Epoch [17/20], Step [11800/12299],final_Loss: 5.1428
Epoch [17/20], Step [11900/12299],final_Loss: 0.2247
Epoch [17/20], Step [12000/12299],final_Loss: 0.5790
Epoch [17/20], Step [12100/12299],final_Loss: 0.2473
Epoch [17/20], Step [12200/12299],final_Loss: 0.2480
Epoch [18/20], Step [100/12299],final_Loss: 0.2393
Epoch [18/20], Step [200/12299],final_Loss: 1.89

Epoch [19/20], Step [1800/12299],final_Loss: 0.1669
Epoch [19/20], Step [1900/12299],final_Loss: 0.5334
Epoch [19/20], Step [2000/12299],final_Loss: 6.4588
Epoch [19/20], Step [2100/12299],final_Loss: 1.2257
Epoch [19/20], Step [2200/12299],final_Loss: 0.4694
Epoch [19/20], Step [2300/12299],final_Loss: 0.8742
Epoch [19/20], Step [2400/12299],final_Loss: 0.2840
Epoch [19/20], Step [2500/12299],final_Loss: 0.4558
Epoch [19/20], Step [2600/12299],final_Loss: 0.2883
Epoch [19/20], Step [2700/12299],final_Loss: 0.6238
Epoch [19/20], Step [2800/12299],final_Loss: 0.9627
Epoch [19/20], Step [2900/12299],final_Loss: 0.2761
Epoch [19/20], Step [3000/12299],final_Loss: 0.7493
Epoch [19/20], Step [3100/12299],final_Loss: 1.1298
Epoch [19/20], Step [3200/12299],final_Loss: 0.6613
Epoch [19/20], Step [3300/12299],final_Loss: 8.6395
Epoch [19/20], Step [3400/12299],final_Loss: 0.2169
Epoch [19/20], Step [3500/12299],final_Loss: 0.3296
Epoch [19/20], Step [3600/12299],final_Loss: 1.0623
Epoch [19/20

Epoch [20/20], Step [5300/12299],final_Loss: 0.7105
Epoch [20/20], Step [5400/12299],final_Loss: 10.2931
Epoch [20/20], Step [5500/12299],final_Loss: 2.0117
Epoch [20/20], Step [5600/12299],final_Loss: 0.6150
Epoch [20/20], Step [5700/12299],final_Loss: 9.7616
Epoch [20/20], Step [5800/12299],final_Loss: 1.1260
Epoch [20/20], Step [5900/12299],final_Loss: 1.1717
Epoch [20/20], Step [6000/12299],final_Loss: 3.9286
Epoch [20/20], Step [6100/12299],final_Loss: 2.1518
Epoch [20/20], Step [6200/12299],final_Loss: 0.9949
Epoch [20/20], Step [6300/12299],final_Loss: 0.3033
Epoch [20/20], Step [6400/12299],final_Loss: 0.4534
Epoch [20/20], Step [6500/12299],final_Loss: 0.3875
Epoch [20/20], Step [6600/12299],final_Loss: 0.9518
Epoch [20/20], Step [6700/12299],final_Loss: 0.7758
Epoch [20/20], Step [6800/12299],final_Loss: 0.5367
Epoch [20/20], Step [6900/12299],final_Loss: 0.9311
Epoch [20/20], Step [7000/12299],final_Loss: 0.4114
Epoch [20/20], Step [7100/12299],final_Loss: 0.6382
Epoch [20/2

In [57]:
#image accuracy 구하기
model.load_state_dict(torch.load('./save_data/total_model.ckpt'))
correct = 0
total = 0
predict=[]
for images, labels in test_loader:
    images = images.cuda()
    labels = labels.cuda()
    _,_,_,_,outputs = model(images)
    predicted = torch.max(softmax(outputs), 1)
    predict_tensor=softmax(outputs)
    predict_numpy=predict_tensor.cpu().detach().numpy()
    predict.append(predict_numpy)
#     print(predicted[1],labels)
    total += labels.size(0)
    correct += (predicted[1] == labels).sum().item()
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


C:\Users\nhs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\nhs\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Test Accuracy of the model on the 10000 test images: 21.22123749898366 %


In [58]:
import csv
dirr="D:/emotion_dataset/mmidataset/VideoWithImageLabels/face_detected/test/labels.csv"
f_read=open(dirr,'r',encoding='utf-8')
reader=csv.reader(f_read)
image_number=[]
image_label=[]
for line in reader:
    
    indexNo_start = line[0].find('image')
    image_start=line[0][indexNo_start+5:]
    image_label.append(line[1])
    indexNo_finish = image_start.find('_')
    image_number.append(int(image_start[:indexNo_finish]))
f_read.close()
image_number.pop()



157

In [59]:

#동영상 갯수
cnt=1
for i in range(len(image_number)):
    if(i<len(image_number)-1):
        if(image_number[i]!=image_number[i+1]):
            
            cnt+=1
cnt+=1
print(len(image_number))
print(cnt)
label_cnt=[]
#image label, 동영상 번호 순으로 csv 만듬
# f = open('predict_label_cnt.csv', 'w', encoding='utf-8', newline='')
# wr = csv.writer(f)
#동영상 갯수
avicnt=1
for i in range(len(image_number)):
    if(i<len(image_number)):
#         wr.writerow([image_label[i],avicnt])
        label_cnt.append([image_label[i],avicnt])
        if(i!=(len(image_number)-1) and image_number[i]!=image_number[i+1]):
            avicnt+=1
        
avicnt+=1
# f.close()
print(avicnt)

12299
155
155


In [60]:
#csv 를 불러와서 predict score와 라벨을 하나의 배열로 묶는다
#그리고 동영상이 바뀔때 까지 더해주고, 바뀌면 평균을 구한다.
# f = open('predict_label_cnt.csv', 'r', encoding='utf-8')
# rdr=csv.reader(f)
#동영상 갯수
prediction_score=0
# lines=[]
# for line in rdr:
#     lines.append([int(line[0]),int(line[1])])
# f.close()
arr=[]
for i in range(len(predict)):
    arr.append([predict[i],label_cnt[i][0],label_cnt[i][1]])
    

In [61]:
#############################################
cnt=1
frame_cnt=0
avi_sum=0
avi_predict=[]
pt_correct=0
pt_notcorrect=0
for i in range(len(arr)):
    
    if(cnt!=arr[i][2] and cnt==arr[i-1][2]):
        avi_sum=avi_sum/frame_cnt
        avi_predict.append(avi_sum)
        if(np.argmax(avi_sum)==int(arr[i-1][1])):
            print("Correct!! -->avi_predict : ",np.argmax(avi_sum),",label : ",arr[i-1][1],",cnt : ",cnt)
            pt_correct+=1
        else:
            print("not Correct!! -->avi_predict : ",np.argmax(avi_sum),",label : ",arr[i-1][1],",cnt : ",cnt)
            pt_notcorrect+=1
        avi_sum=0
        cnt+=1
        frame_cnt=0
        
    else:
        avi_sum+=arr[i][0]
        frame_cnt+=1
frame_cnt=0
avi_sum=0
for i in range(len(arr)):
    if (arr[i][2]==avicnt-1):
        avi_sum+=arr[i][0]
        frame_cnt+=1
avi_sum=avi_sum/frame_cnt
avi_predict.append(avi_sum)
if(np.argmax(avi_sum)==int(arr[len(arr)-1][1])):
    print("Correct!! -->avi_predict : ",np.argmax(avi_sum),",label : ",arr[len(arr)-1][1],",cnt : ",cnt)
    pt_correct+=1
else:
    print("not Correct!! -->avi_predict : ",np.argmax(avi_sum),",label : ",arr[len(arr)-1][1],",cnt : ",cnt)
    pt_notcorrect+=1
# #라벨과 동영상 predict 비교
# cnt=1
# correct = 0
# for i in range(len(arr)):
#     if(cnt!=arr[i][2] and cnt==arr[i-1][2]):
# #         print(np.argmax(avi_predict[cnt]),arr[i-1][1])
#         if(np.argmax(avi_predict[cnt])==arr[i-1][1]):
#              correct+=1
#         cnt+=1

# for i in range(len(arr)):
#     if (arr[i][2]==208):
# #         print((np.argmax(avi_predict[cnt-1]),arr[i][1]))
#         if(np.argmax(avi_predict[cnt-1])==arr[i][1]):
#             correct+=1
#         break
total = avicnt
print("total : ",total,",correct : ",pt_correct,",not correct : ",pt_notcorrect)
print("total avi files : ",cnt)
print("accuracy : ",pt_correct/total*100)

not Correct!! -->avi_predict :  3 ,label :  1 ,cnt :  1
not Correct!! -->avi_predict :  3 ,label :  1 ,cnt :  2
not Correct!! -->avi_predict :  4 ,label :  2 ,cnt :  3
not Correct!! -->avi_predict :  4 ,label :  3 ,cnt :  4
Correct!! -->avi_predict :  3 ,label :  3 ,cnt :  5
not Correct!! -->avi_predict :  3 ,label :  4 ,cnt :  6
Correct!! -->avi_predict :  4 ,label :  4 ,cnt :  7
not Correct!! -->avi_predict :  4 ,label :  5 ,cnt :  8
not Correct!! -->avi_predict :  3 ,label :  5 ,cnt :  9
not Correct!! -->avi_predict :  3 ,label :  6 ,cnt :  10
not Correct!! -->avi_predict :  4 ,label :  6 ,cnt :  11
not Correct!! -->avi_predict :  4 ,label :  2 ,cnt :  12
not Correct!! -->avi_predict :  4 ,label :  2 ,cnt :  13
not Correct!! -->avi_predict :  4 ,label :  3 ,cnt :  14
not Correct!! -->avi_predict :  5 ,label :  4 ,cnt :  15
Correct!! -->avi_predict :  4 ,label :  4 ,cnt :  16
not Correct!! -->avi_predict :  4 ,label :  1 ,cnt :  17
not Correct!! -->avi_predict :  4 ,label :  3 ,cnt :

not Correct!! -->avi_predict :  4 ,label :  1 ,cnt :  146
not Correct!! -->avi_predict :  4 ,label :  1 ,cnt :  147
not Correct!! -->avi_predict :  4 ,label :  1 ,cnt :  148
not Correct!! -->avi_predict :  4 ,label :  2 ,cnt :  149
not Correct!! -->avi_predict :  4 ,label :  3 ,cnt :  150
not Correct!! -->avi_predict :  4 ,label :  3 ,cnt :  151
Correct!! -->avi_predict :  4 ,label :  4 ,cnt :  152
Correct!! -->avi_predict :  4 ,label :  4 ,cnt :  153
Correct!! -->avi_predict :  4 ,label :  4 ,cnt :  154
total :  155 ,correct :  33 ,not correct :  121
total avi files :  154
accuracy :  21.29032258064516
